# RDFLib

>[RDFLib](https://rdflib.readthedocs.io/) 是一个纯 Python 包，用于处理 [RDF](https://en.wikipedia.org/wiki/Resource_Description_Framework)。`RDFLib` 包含了处理 `RDF` 所需的大部分内容，包括：
>- RDF/XML、N3、NTriples、N-Quads、Turtle、TriX、Trig 和 JSON-LD 的解析器和序列化器
>- 一个 Graph 接口，可以由多种 Store 实现提供支持
>- 用于内存、持久磁盘（Berkeley DB）和远程 SPARQL 端点的 Store 实现
>- 一个 SPARQL 1.1 实现 - 支持 SPARQL 1.1 查询和更新语句
>- SPARQL 函数扩展机制

图数据库是基于网络状模型的应用程序的绝佳选择。为了标准化此类图的语法和语义，W3C 推荐使用 `Semantic Web Technologies`，参见 [Semantic Web](https://www.w3.org/standards/semanticweb/)。

[SPARQL](https://www.w3.org/TR/sparql11-query/) 作为这些图的查询语言，类似于 `SQL` 或 `Cypher`。本笔记通过生成 `SPARQL` 来演示如何将 LLM 用作图数据库的自然语言接口。

**免责声明：** 截至目前，通过 LLM 生成 `SPARQL` 查询仍然不稳定。请特别注意会修改图的 `UPDATE` 查询。

## 设置

我们必须安装一个python库：

In [ ]:
!pip install rdflib

您可以针对多种来源运行查询，包括网络上的文件、本地可用的文件、SPARQL 端点（例如 [Wikidata](https://www.wikidata.org/wiki/Wikidata:Main_Page)）以及 [三元组存储](https://www.w3.org/wiki/LargeTripleStores)。

In [3]:
from langchain.chains import GraphSparqlQAChain
from langchain_community.graphs import RdfGraph
from langchain_openai import ChatOpenAI

In [4]:
graph = RdfGraph(
    source_file="http://www.w3.org/People/Berners-Lee/card",
    standard="rdf",
    local_copy="test.ttl",
)

请注意，如果源是只读的，则必须提供 `local_file` 以将更改存储在本地。

## 刷新图谱模式信息
如果数据库的模式发生变化，您可以刷新生成 SPARQL 查询所需的模式信息。

In [5]:
graph.load_schema()

In [6]:
graph.get_schema

In the following, each IRI is followed by the local name and optionally its description in parentheses. 
The RDF graph supports the following node types:
<http://xmlns.com/foaf/0.1/PersonalProfileDocument> (PersonalProfileDocument, None), <http://www.w3.org/ns/auth/cert#RSAPublicKey> (RSAPublicKey, None), <http://www.w3.org/2000/10/swap/pim/contact#Male> (Male, None), <http://xmlns.com/foaf/0.1/Person> (Person, None), <http://www.w3.org/2006/vcard/ns#Work> (Work, None)
The RDF graph supports the following relationships:
<http://www.w3.org/2000/01/rdf-schema#seeAlso> (seeAlso, None), <http://purl.org/dc/elements/1.1/title> (title, None), <http://xmlns.com/foaf/0.1/mbox_sha1sum> (mbox_sha1sum, None), <http://xmlns.com/foaf/0.1/maker> (maker, None), <http://www.w3.org/ns/solid/terms#oidcIssuer> (oidcIssuer, None), <http://www.w3.org/2000/10/swap/pim/contact#publicHomePage> (publicHomePage, None), <http://xmlns.com/foaf/0.1/openid> (openid, None), <http://www.w3.org/ns/pim/space#storage> (

## 查询图谱

现在，您可以使用图谱 SPARQL QA 链来询问图谱相关的问题。

In [ ]:
chain = GraphSparqlQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True
)

In [12]:
chain.run("What is Tim Berners-Lee's work homepage?")



> Entering new GraphSparqlQAChain chain...
Identified intent:
SELECT
Generated SPARQL:
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
SELECT ?homepage
WHERE {
    ?person foaf:name "Tim Berners-Lee" .
    ?person foaf:workplaceHomepage ?homepage .
}
Full Context:
[]

> Finished chain.


"Tim Berners-Lee's work homepage is http://www.w3.org/People/Berners-Lee/."

## 更新图

类似地，您也可以使用自然语言更新图，即插入三元组。

In [14]:
chain.run(
    "Save that the person with the name 'Timothy Berners-Lee' has a work homepage at 'http://www.w3.org/foo/bar/'"
)



> Entering new GraphSparqlQAChain chain...
Identified intent:
UPDATE
Generated SPARQL:
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
INSERT {
    ?person foaf:workplaceHomepage <http://www.w3.org/foo/bar/> .
}
WHERE {
    ?person foaf:name "Timothy Berners-Lee" .
}

> Finished chain.


'Successfully inserted triples into the graph.'

让我们来验证一下结果：

In [27]:
query = (
    """PREFIX foaf: <http://xmlns.com/foaf/0.1/>\n"""
    """SELECT ?hp\n"""
    """WHERE {\n"""
    """    ?person foaf:name "Timothy Berners-Lee" . \n"""
    """    ?person foaf:workplaceHomepage ?hp .\n"""
    """}"""
)
graph.query(query)

[(rdflib.term.URIRef('https://www.w3.org/'),),
 (rdflib.term.URIRef('http://www.w3.org/foo/bar/'),)]

## 返回 SPARQL 查询
您可以使用 `return_sparql_query` 参数从 Sparql QA Chain 返回 SPARQL 查询步骤。

In [20]:
chain = GraphSparqlQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True, return_sparql_query=True
)

In [29]:
result = chain("What is Tim Berners-Lee's work homepage?")
print(f"SPARQL query: {result['sparql_query']}")
print(f"Final answer: {result['result']}")



> Entering new GraphSparqlQAChain chain...
Identified intent:
SELECT
Generated SPARQL:
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
SELECT ?workHomepage
WHERE {
    ?person foaf:name "Tim Berners-Lee" .
    ?person foaf:workplaceHomepage ?workHomepage .
}
Full Context:
[]

> Finished chain.
SPARQL query: PREFIX foaf: <http://xmlns.com/foaf/0.1/>
SELECT ?workHomepage
WHERE {
    ?person foaf:name "Tim Berners-Lee" .
    ?person foaf:workplaceHomepage ?workHomepage .
}
Final answer: Tim Berners-Lee's work homepage is http://www.w3.org/People/Berners-Lee/.


In [30]:
print(result["sparql_query"])

PREFIX foaf: <http://xmlns.com/foaf/0.1/>
SELECT ?workHomepage
WHERE {
    ?person foaf:name "Tim Berners-Lee" .
    ?person foaf:workplaceHomepage ?workHomepage .
}
